<a href="https://colab.research.google.com/github/orhansonmeztr/Bio-LLMs/blob/main/fine_tuning_Qwen3_0_6b_4bit_with_turkish_mmlu_6k_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print("\n=== 2) PyTorch & GPU ===")
print("torch            :", torch.__version__)
print("CUDA available   :", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU              :", torch.cuda.get_device_name(0))


=== 2) PyTorch & GPU ===
torch            : 2.6.0+cu124
CUDA available   : True
GPU              : NVIDIA A100-SXM4-40GB


In [2]:
!pip install -q fsspec==2023.9.2 transformers==4.53.2 tokenizers==0.21.2 accelerate==1.8.1 peft==0.16.0 \
                bitsandbytes==0.46.1 datasets==4.0.0 evaluate==0.4.5 huggingface_hub==0.33.4 gcsfs==2023.9.2

In [3]:
# !pip install -q fsspec==2023.9.2 transformers tokenizers accelerate peft \
#                 bitsandbytes datasets evaluate huggingface_hub gcsfs

In [4]:
!pip install -q -U datasets>=2.21.0

In [5]:
# !pip list

In [6]:
# =========================
# # Testing the installed libraries.
# =========================
import importlib, torch, textwrap

# Kontrol edilecek temel paketler (sürüm beklentileri opsiyonel)
required = {
    "transformers":  "4.53.2",
    "datasets":      "4.0.0",
    "accelerate":    "1.8.1",
    "peft":          "0.16.0",
    "bitsandbytes":  "0.46.1",
    "tokenizers":    "0.21.2",
    "evaluate":      "0.4.5",
    "fsspec":        "2023.9.2",
    "gcsfs":         "2023.9.2",
}

print("\n=== 1) Sürüm kontrolleri ===")
for name, wanted in required.items():
    try:
        mod = importlib.import_module(name)
        ver = mod.__version__
        ok  = wanted is None or ver == wanted
        status = "✅" if ok else f"⚠️ (beklenen {wanted})"
        print(f"{name:15s} {ver:10s} {status}")
    except Exception as e:
        print(f"{name:15s} YÜKLENEMEDİ → {e}")

print("\n=== 2) PyTorch & GPU ===")
print("torch            :", torch.__version__)
print("CUDA available   :", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU              :", torch.cuda.get_device_name(0))

print("\n=== 3) Mini transformers testi ===")
try:
    from transformers import AutoTokenizer, AutoModelForCausalLM
    model_id = "sshleifer/tiny-gpt2"     # 15 MB, hızlı indirilir
    tok = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
    out = model(**tok("Merhaba", return_tensors="pt").to(model.device))
    print("Çıktı boyutu      :", tuple(out.logits.shape))
    print("✅ Transformers ileri geçişi başarılı")
except Exception as e:
    print("❌ Transformers testi başarısız →", e)

print("\n=== 4) datasets okuma testi ===")
try:
    from datasets import Dataset
    ds = Dataset.from_dict({"x":[1,2,3]})
    print(ds)
    print("✅ datasets modülü çalışıyor")
except Exception as e:
    print("❌ datasets testi başarısız →", e)



=== 1) Sürüm kontrolleri ===
transformers    4.53.2     ✅
datasets        4.0.0      ✅
accelerate      1.8.1      ✅
peft            0.16.0     ✅
bitsandbytes    0.46.1     ✅
tokenizers      0.21.2     ✅
evaluate        0.4.5      ✅
fsspec          2023.9.2   ✅
gcsfs           2023.9.2   ✅

=== 2) PyTorch & GPU ===
torch            : 2.6.0+cu124
CUDA available   : True
GPU              : NVIDIA A100-SXM4-40GB

=== 3) Mini transformers testi ===
Çıktı boyutu      : (1, 3, 50257)
✅ Transformers ileri geçişi başarılı

=== 4) datasets okuma testi ===
Dataset({
    features: ['x'],
    num_rows: 3
})
✅ datasets modülü çalışıyor


In [7]:
import torch
import gc
import os

# Bellek optimizasyonu için environment variables
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Cache temizliği
torch.cuda.empty_cache()
gc.collect()

609

In [8]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/turkish_mmlu_qwen3

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/turkish_mmlu_qwen3


In [9]:
import time
import re

_pat = re.compile(r"'([^']+)'")

def explode(row):
    if isinstance(row["secenekler"], list):
        opts = row["secenekler"]
    else:
        opts = _pat.findall(row["secenekler"])
    for i, opt in enumerate(opts):
        row[f"secenek_{i}"] = opt.strip()
    return row

In [10]:
# 2) Veri setini yükle  ----------------------------------------------------
from datasets import load_dataset, DatasetDict

files = {
    "train": "/content/drive/MyDrive/turkish_mmlu_qwen3/train-00000-of-00001.csv",
    "test": "/content/drive/MyDrive/turkish_mmlu_qwen3/test-00000-of-00001.csv",
    }
ds_raw = load_dataset("csv", data_files=files)

ds = ds_raw.map(explode, num_proc=4)

subset_size = 1_000
train_full   = ds["train"].shuffle(seed=42).select(range(6000))
test_sub   = ds["test"].shuffle(seed=42).select(range(1000))
train_val_split = train_full.train_test_split(test_size=0.05, seed=42)

final_ds = DatasetDict({
    'train': train_val_split['train'],      # ~5,700 örnek
    'validation': train_val_split['test'],  # ~300 örnek
    'test': test_sub                        # 1,000 örnek (son test için)
})

In [11]:
final_ds["train"][10]

{'bolum': 'Bankacılık ve Sigortacılık',
 'konu': 'Finansal Yönetim I',
 'soru': 'Ahdî tarafından yazılan “Gülşen-i Şuara” adlı tezkirenin edebiyat tarihi açısından asıl önemi aşağıdakilerden hangisidir?',
 'cevap': 2,
 'aciklama': None,
 'secenekler': "['Önceki tezkirecilerin vermediği şairleri ele alması'\n 'Padişahları ve yüksek zümreden kişileri anlatırken dilinin sade olması'\n 'Bağdat ve çevresinde yaşayan şairleri anlatan tek kaynak durumunda olması'\n 'Mukaddime ve ravza adı verilen bölümlere ayrılması'\n 'Bazı şairler hakkındaki bilgilerin diğer tezkirelerden alınması']",
 'secenek_0': 'Önceki tezkirecilerin vermediği şairleri ele alması',
 'secenek_1': 'Padişahları ve yüksek zümreden kişileri anlatırken dilinin sade olması',
 'secenek_2': 'Bağdat ve çevresinde yaşayan şairleri anlatan tek kaynak durumunda olması',
 'secenek_3': 'Mukaddime ve ravza adı verilen bölümlere ayrılması',
 'secenek_4': 'Bazı şairler hakkındaki bilgilerin diğer tezkirelerden alınması'}

In [12]:
# ============================================================
# Qwen3-0.6B – Fine-tuning ÖNCESİ Sıfır-Shot Değerlendirme
# ============================================================
import numpy as np
import math, torch, evaluate
from tqdm.auto import tqdm
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          LogitsProcessor, LogitsProcessorList)

model_id   = "Qwen/Qwen3-0.6B"

# 1) Tokenizer & model  ----------------------------------------------------
tok = AutoTokenizer.from_pretrained(model_id,
                                    trust_remote_code=True,
                                    padding_side="left")
tok.pad_token = tok.eos_token          # Qwen3'te pad yoksa eos kullan

model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True)

In [13]:
# Eğer secenek_0 ... secenek_4 kolonları yoksa önce explode() fonksiyonu uygulanmalı
# def format_row(row):
#     parts = [f"Soru: {row['soru']}"]
#     i = 0
#     parts.append("<secenekler>")
#     while f"secenek_{i}" in row:
#         parts.append(f"{i}) {row[f'secenek_{i}']}")
#         i += 1
#     parts.append("</secenekler>")
#     parts.append("\Yalnızca doğru seçeneğin numarasını (0,1,2,3,4) cevap olarak ver.")
#     parts.append("\nCevap:")
#     # parts.append("Yalnızca doğru seçeneğin numarasını (0-4) yaz:")
#     return "\n".join(parts)

def format_prompt(row):
    parts = [f"Soru: {row['soru']}"]
    i = 0
    parts.append("\n<secenekler>")
    while f"secenek_{i}" in row and row[f'secenek_{i}'] is not None:
        parts.append(f"{i}) {row[f'secenek_{i}']}")
        i += 1
    parts.append("</secenekler>")
    # Talimatı net ve basit tutuyoruz.
    parts.append("\nCevap:")
    return "\n".join(parts)

# def to_chat_for_testing(prompt: str):
#     msgs = [
#         {"role": "user",      "content": prompt},
#         {"role": "assistant", "content": ""}
#     ]
#     return tok.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)

# def to_chat_for_testing(r):
#     prompt = format_row(r)
#     msgs   = [
#         {"role": "user",      "content": prompt},
#     ]
#     return {"text": tok.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)}

# def to_chat_for_training(r):
#     prompt = format_row(r)
#     msgs   = [
#         {"role": "user",      "content": prompt},
#         {"role": "assistant", "content": str(r["cevap"])}
#     ]
#     return {"text": tok.apply_chat_template(msgs, tokenize=False, add_generation_prompt=False)}


# Bu fonksiyon, EĞİTİM için tam bir diyalog metni oluşturur.
def create_training_text(row):
    prompt = format_prompt(row)
    answer = str(row["cevap"])

    # Chat template'i manuel olarak oluşturarak <think> etiketinden kaçınıyoruz.
    # Bu en güvenilir yöntemdir.
    chat_text = (
        f"<|im_start|>user\n{prompt}<|im_end|>\n"
        f"<|im_start|>assistant\n{answer}<|im_end|>"
    )
    return {"text": chat_text}

# Bu fonksiyon, DEĞERLENDİRME için modelin tamamlayacağı bir metin oluşturur.
def create_test_text(row):
    prompt = format_prompt(row)
    msgs = [{"role": "user", "content": prompt}]
    chat_text = tok.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    return {"text": chat_text}

digit_ids = [tok(str(i)).input_ids[0] for i in range(5)]

class OnlyDigits04(LogitsProcessor):
    def __call__(self, input_ids: torch.Tensor, scores: torch.Tensor):
        mask = torch.full_like(scores, -float("inf"))
        mask[:, digit_ids] = 0
        return scores + mask

processors = LogitsProcessorList([OnlyDigits04()])

In [14]:
train_data = final_ds['train'].map(create_training_text, remove_columns=final_ds['train'].column_names)
validation_data = final_ds['validation'].map(create_training_text, remove_columns=final_ds['validation'].column_names)
test_data = final_ds['test'].map(create_test_text, remove_columns=final_ds['test'].column_names)

Map:   0%|          | 0/5700 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [15]:
metric = evaluate.load("accuracy")

def evaluate_model_on_test_set(model, test_prompts_dataset, reference_answers, name="Model"):
    batch_size = 32
    chat_prompts = test_prompts_dataset["text"]
    total_batches = math.ceil(len(chat_prompts) / batch_size)

    for i in tqdm(range(total_batches), desc=f"Evaluating {name}", ncols=80):
        lo, hi = i*batch_size, (i+1)*batch_size

        batch = tok(chat_prompts[lo:hi], return_tensors="pt", padding=True).to(model.device)

        out = model.generate(**batch,
                             logits_processor=processors,
                             max_new_tokens=1,
                             eos_token_id=tok.eos_token_id,
                             pad_token_id=tok.pad_token_id)

        generated_tokens = out[:, batch.input_ids.shape[1]:]
        preds_str = tok.batch_decode(generated_tokens, skip_special_tokens=True)

        preds = []
        for p_str in preds_str:
            try:
                preds.append(int(p_str.strip()))
            except ValueError:
                preds.append(-1)

        # Referans cevapları artık doğrudan parametre olarak gelen listeden alıyoruz.
        refs  = [int(x) for x in reference_answers[lo:hi]]
        print("preds:", preds, ", refs", refs)
        metric.add_batch(predictions=preds, references=refs)

    return metric.compute()["accuracy"]

# def compute_metrics(eval_preds):
#     logits, labels = eval_preds
#     predictions = np.argmax(logits, axis=-1)

#     # -100 etiketli token'ları (padding) hesaptan çıkar
#     mask = labels != -100

#     return metric.compute(predictions=predictions[mask], references=labels[mask])

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    mask = labels != -100
    active_labels = labels[mask]
    active_logits = logits[mask]
    active_predictions = np.argmax(active_logits, axis=-1)
    return metric.compute(predictions=active_predictions, references=active_labels)

In [16]:
acc_before_ft = evaluate_model_on_test_set(model, test_data, test_sub["cevap"], "Base Model")
print(f"Accuracy before fine-tuning: {acc_before_ft:.3f}")

# acc_before_ft = evaluate_model_on_test_set(model, "Base Model")
# print(f"Accuracy before fine-tuning: {acc_before_ft:.3f}")

Evaluating Base Model:   0%|                             | 0/32 [00:00<?, ?it/s]

preds: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] , refs [3, 2, 2, 1, 3, 2, 3, 1, 4, 3, 1, 0, 4, 1, 3, 4, 4, 0, 1, 4, 0, 4, 1, 1, 4, 2, 0, 4, 3, 3, 0, 2]
preds: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] , refs [0, 0, 0, 3, 1, 4, 3, 2, 1, 4, 1, 1, 2, 2, 3, 1, 0, 1, 1, 1, 1, 0, 3, 4, 3, 4, 0, 3, 4, 3, 0, 1]
preds: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0] , refs [1, 1, 0, 4, 3, 4, 2, 0, 3, 3, 4, 0, 1, 0, 2, 3, 0, 0, 3, 4, 1, 0, 0, 3, 3, 4, 0, 0, 2, 1, 1, 2]
preds: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] , refs [4, 0, 0, 2, 3, 1, 3, 2, 3, 3, 1, 3, 2, 3, 1, 3, 0, 0, 0, 0, 3, 2, 1, 2, 2, 0, 3, 1, 3, 1, 3, 0]
preds: [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] , refs [3, 2, 0, 2, 3, 3, 2, 0, 2, 1, 4, 4, 3, 3, 1, 3, 2, 4, 3,

In [17]:
# %%time
# from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
# import gc

# # ==============================================================================
# # 1. BELLEK YÖNETİMİ (İsteğe Bağlı ama Faydalı)
# # ==============================================================================
# # Bellek optimizasyonu için environment variables
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# # Cache temizliği
# torch.cuda.empty_cache()
# gc.collect()

# # ==============================================================================
# # 2. VERİ SETİNİ TOKENIZE ETME
# # ==============================================================================
# def tokenize(batch):
#     return tok(batch["text"],
#                truncation=True,
#                max_length=1024,
#                padding="max_length")

# train_tok = train_data.map(tokenize, batched=True, remove_columns=["text"])
# validation_tok = validation_data.map(tokenize, batched=True, remove_columns=["text"])

# train_tok.set_format(type="torch")
# validation_tok.set_format(type="torch")

# collator = DataCollatorForLanguageModeling(tok, mlm=False)

# # ==============================================================================
# # 3. EĞİTİM ARGÜMANLARI (ACCURACY OLMADAN)
# # ==============================================================================
# per_device_train_batch_size = 8
# gradient_accumulation_steps = 16
# effective_batch_size = per_device_train_batch_size * gradient_accumulation_steps
# steps_per_epoch = len(train_tok) // effective_batch_size

# args = TrainingArguments(
#     output_dir="qwen3_0.6b_turkish_mmlu_5k_lora_4bit_loss_only",
#     num_train_epochs=3,
#     per_device_train_batch_size=per_device_train_batch_size,
#     gradient_accumulation_steps=gradient_accumulation_steps,
#     learning_rate=5e-5,
#     bf16=True,

#     eval_strategy="epoch",
#     save_strategy="epoch",

#     load_best_model_at_end=True,
#     metric_for_best_model="loss",
#     greater_is_better=False,

#     logging_steps=100,
#     save_total_limit=2,
#     report_to="none",
#     remove_unused_columns=False,
#     gradient_checkpointing=True,
# )

# # ==============================================================================
# # 4. TRAINER'I OLUŞTURMA VE EĞİTİM (compute_metrics OLMADAN)
# # ==============================================================================
# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=train_tok,
#     eval_dataset=validation_tok,
#     data_collator=collator
#     # compute_metrics parametresi KASITLI OLARAK EKLENMEDİ
# )

# trainer.train()

In [18]:
%%time
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os
import gc
import torch

# Bellek yönetimi
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()
gc.collect()

from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training # <-- YENİ İMPORT

# Base modeli 4-bit olarak yükle
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    trust_remote_code=True
)

# ==============================================================================
# KRİTİK DÜZELTME: Modeli k-bit eğitim için hazırla
# ==============================================================================
# Bu satır, gradient checkpointing'in 4-bit modelle uyumlu çalışmasını sağlar
# ve kaydetme sorunlarını çözer.
base_model = prepare_model_for_kbit_training(base_model)
# ==============================================================================

# LoRA yapılandırmasını oluştur
lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

# Hazırlanmış base model üzerine LoRA'yı uygula
model = get_peft_model(base_model, lora_cfg)

# Eğitilebilir parametreleri yazdır
model.print_trainable_parameters()

def tokenize(batch):
    return tok(batch["text"],
               truncation=True,
               max_length=1024,
               padding="max_length")

train_tok = train_data.map(tokenize, batched=True, remove_columns=["text"])
validation_tok = validation_data.map(tokenize, batched=True, remove_columns=["text"])

train_tok.set_format(type="torch")
validation_tok.set_format(type="torch")

collator = DataCollatorForLanguageModeling(tok, mlm=False)

# Bir epoch'taki adım sayısını hesaplayalım
per_device_train_batch_size = 8
gradient_accumulation_steps = 16
effective_batch_size = per_device_train_batch_size * gradient_accumulation_steps
steps_per_epoch = len(train_tok) // effective_batch_size

print(f"Eğitim veri seti boyutu: {len(train_tok)}")
print(f"Etkin batch boyutu: {effective_batch_size}")
print(f"Bir epoch'taki adım sayısı: {steps_per_epoch}")

args = TrainingArguments(
    output_dir="qwen3_0.6b_turkish_mmlu_6k_lora_4bit_loss_only",
    num_train_epochs=8,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=2e-5,
    bf16=True,
    do_eval=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    report_to="none",
    remove_unused_columns=False,
    gradient_checkpointing=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=validation_tok,
    data_collator=collator
)

# Eğitimi başlat
trainer.train()

# ==============================================================================
# KRİTİK DÜZELTME: En iyi modeli diske kaydet!
# ==============================================================================
print("Eğitim tamamlandı. En iyi model (en düşük validation loss'a sahip olan) kaydediliyor...")
trainer.save_model()
print("Model başarıyla kaydedildi.")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 4,587,520 || all params: 600,637,440 || trainable%: 0.7638


Map:   0%|          | 0/5700 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Eğitim veri seti boyutu: 5700
Etkin batch boyutu: 128
Bir epoch'taki adım sayısı: 44


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,No log,2.589553
2,No log,2.374081
3,No log,2.321649
4,No log,2.290340
5,No log,2.269888
6,No log,2.256450
7,No log,2.248973
8,No log,2.246403


Eğitim tamamlandı. En iyi model (en düşük validation loss'a sahip olan) kaydediliyor...
Model başarıyla kaydedildi.
CPU times: user 1h 10min, sys: 10min 23s, total: 1h 20min 23s
Wall time: 1h 20min 16s


In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import os

# ==============================================================================
# 1. EN İYİ CHECKPOINT'İN YOLUNU BUL
# ==============================================================================
# Trainer, en iyi checkpoint'in yolunu kendi state'inde tutar.
best_checkpoint_path = trainer.state.best_model_checkpoint
print(f"En iyi checkpoint bulundu: {best_checkpoint_path}")

# ==============================================================================
# 2. EN İYİ ADAPTÖRÜ MANUEL OLARAK YÜKLE
# ==============================================================================
# Önce base modeli tekrar yükleyelim (temiz bir başlangıç için)
base_id = "Qwen/Qwen3-0.6B"
tok = AutoTokenizer.from_pretrained(base_id, trust_remote_code=True, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(base_id, device_map="auto", torch_dtype="auto")

# Şimdi, base modelin üzerine en iyi checkpoint'teki adaptörü yüklüyoruz.
# Bu komut, 'adapter_config.json' dosyasını checkpoint klasörünün içinde bulacaktır.
print("En iyi adaptör base model üzerine yükleniyor...")
model = PeftModel.from_pretrained(model, best_checkpoint_path)
print("Adaptör başarıyla yüklendi.")

# ==============================================================================
# 3. BİRLEŞTİRME VE KAYDETME
# ==============================================================================
# Modelleri birleştir
print("Adaptör ve base model birleştiriliyor...")
merged = model.merge_and_unload()
print("Birleştirme tamamlandı.")

# Birleştirilmiş modeli YENİ bir klasöre kaydet
merged_model_path = "qwen3_0.6b_turkish_mmlu_6k_lora_merged" # Karışmaması için yeni isim
print(f"Birleştirilmiş model '{merged_model_path}' klasörüne kaydediliyor...")

merged.save_pretrained(
    merged_model_path,
    safe_serialization=True
)
tok.save_pretrained(merged_model_path)
print("Kaydetme başarıyla tamamlandı!")

En iyi checkpoint bulundu: qwen3_0.6b_turkish_mmlu_6k_lora_4bit_loss_only/checkpoint-360
En iyi adaptör base model üzerine yükleniyor...
Adaptör başarıyla yüklendi.
Adaptör ve base model birleştiriliyor...
Birleştirme tamamlandı.
Birleştirilmiş model 'qwen3_0.6b_turkish_mmlu_6k_lora_merged' klasörüne kaydediliyor...
Kaydetme başarıyla tamamlandı!


In [20]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from peft import PeftModel

# base_id   = "Qwen/Qwen3-0.6B"

# # 1. ADAPTÖR YOLU: Trainer'ın çıktı klasörünü belirtin.
# # Bu, eğitim sırasında 'output_dir' olarak verdiğiniz yoldur.
# adapter_path = "qwen3_0.6b_turkish_mmlu_5k_lora_4bit_loss_only"

# # 2. BİRLEŞTİRİLMİŞ MODELİN KAYDEDİLECEĞİ YOL: Yeni ve farklı bir isim verin.
# merged_model_path = "qwen3_0.6b_turkish_mmlu_5k_lora_4bit_loss_only_merged"

# # Tokenizer'ı ve base modeli yükle
# tok   = AutoTokenizer.from_pretrained(base_id, trust_remote_code=True, padding_side="left")
# model = AutoModelForCausalLM.from_pretrained(base_id, device_map="auto", torch_dtype="auto")

# # Doğru yoldan adaptörü yükle
# model = PeftModel.from_pretrained(model, adapter_path)

# # Modelleri birleştir
# merged = model.merge_and_unload()

# # Yeni yola kaydet
# print(f"Birleştirilmiş model '{merged_model_path}' klasörüne kaydediliyor...")
# merged.save_pretrained(
#     merged_model_path,
#     safe_serialization=True
# )
# tok.save_pretrained(merged_model_path)
# print("Kaydetme tamamlandı.")

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer

merged_dir = "qwen3_0.6b_turkish_mmlu_6k_lora_merged"    # model.safetensors'in bulunduğu klasör
tok  = AutoTokenizer.from_pretrained(merged_dir, trust_remote_code=True,
                                     padding_side="left")
ft_model = AutoModelForCausalLM.from_pretrained(
    merged_dir,
    torch_dtype="auto",
    device_map="auto"
)

# =========================================================
# 2) Test doğruluğunu ölç
# =========================================================
# acc_after_ft = evaluate_model_on_test_set(ft_model, "LoRA-1k Model")
# print(f"Accuracy after fine-tuning: {acc_after_ft:.3f}")

acc_after_ft = evaluate_model_on_test_set(ft_model, test_data, test_sub["cevap"], "LoRA-6k Model")
print(f"Accuracy after fine-tuning: {acc_after_ft:.3f}")


Evaluating LoRA-5k Model:   0%|                          | 0/32 [00:00<?, ?it/s]

preds: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] , refs [3, 2, 2, 1, 3, 2, 3, 1, 4, 3, 1, 0, 4, 1, 3, 4, 4, 0, 1, 4, 0, 4, 1, 1, 4, 2, 0, 4, 3, 3, 0, 2]
preds: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] , refs [0, 0, 0, 3, 1, 4, 3, 2, 1, 4, 1, 1, 2, 2, 3, 1, 0, 1, 1, 1, 1, 0, 3, 4, 3, 4, 0, 3, 4, 3, 0, 1]
preds: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] , refs [1, 1, 0, 4, 3, 4, 2, 0, 3, 3, 4, 0, 1, 0, 2, 3, 0, 0, 3, 4, 1, 0, 0, 3, 3, 4, 0, 0, 2, 1, 1, 2]
preds: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] , refs [4, 0, 0, 2, 3, 1, 3, 2, 3, 3, 1, 3, 2, 3, 1, 3, 0, 0, 0, 0, 3, 2, 1, 2, 2, 0, 3, 1, 3, 1, 3, 0]
preds: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] , refs [3, 2, 0, 2, 3, 3, 2, 0, 2, 1, 4, 4, 3, 3, 1, 3, 2, 4, 3,

In [9]:
from google.colab import userdata
from huggingface_hub import HfApi, create_repo, login
import os

# ==============================================================================
# ADIM 1: GİRİŞ YAP
# ==============================================================================
try:
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token)
    print("Hugging Face'e Colab Secrets kullanılarak başarıyla giriş yapıldı.")
except Exception as e:
    print(f"Giriş sırasında bir hata oluştu: {e}")
    print("Lütfen Colab Secrets'de 'HF_TOKEN' anahtarının doğru ayarlandığından emin olun.")

# ==============================================================================
# ADIM 2: TUTARLI DEĞİŞKENLERİ TANIMLA
# ==============================================================================

# --- Ayarlarınızı SADECE BURADA Değiştirin ---
hf_username = "orhansonmeztr"
# Hugging Face Hub'da olmasını istediğiniz TEK ve NİHAİ depo adı:
repo_name = "qwen3-0.6b-4bit-turkish-mmlu-lora"
# Yereldeki birleştirilmiş modelinizin klasör adı:
local_model_path = "/content/drive/MyDrive/turkish_mmlu_qwen3/qwen3_0.6b_turkish_mmlu_6k_lora_merged"
# ------------------------------------------------

# repo_id'yi bu tutarlı değişkenlerden oluştur
repo_id = f"{hf_username}/{repo_name}"

# ==============================================================================
# ADIM 3: DEPOYU OLUŞTUR VE YÜKLE
# ==============================================================================
try:
    # Depoyu oluştur
    print(f"'{repo_id}' deposu oluşturuluyor/kontrol ediliyor...")
    create_repo(repo_id, private=True, exist_ok=True)

    # API ile dosyaları yükle
    api = HfApi()

    print(f"'{local_model_path}' klasöründeki dosyalar '{repo_id}' deposuna yükleniyor...")

    api.upload_folder(
        folder_path=local_model_path,
        repo_id=repo_id, # <-- Artık create_repo ile aynı repo_id'yi kullanıyor
        repo_type="model",
        commit_message="Fine-tuned Qwen3-0.6B-4bit on Turkish MMLU (LoRA merged)"
    )

    print("Yükleme başarıyla tamamlandı!")
    print(f"Modelinize şu adresten erişebilirsiniz: https://huggingface.co/{repo_id}")

except Exception as e:
    print(f"Yükleme sırasında bir hata oluştu: {e}")

Hugging Face'e Colab Secrets kullanılarak başarıyla giriş yapıldı.
'orhansonmeztr/qwen3-0.6b-4bit-turkish-mmlu-lora' deposu oluşturuluyor/kontrol ediliyor...
'/content/drive/MyDrive/turkish_mmlu_qwen3/qwen3_0.6b_turkish_mmlu_6k_lora_merged' klasöründeki dosyalar 'orhansonmeztr/qwen3-0.6b-4bit-turkish-mmlu-lora' deposuna yükleniyor...
Yükleme sırasında bir hata oluştu: (Request ID: Root=1-687fc13d-77e94ec8187892e143d8bc13;2a3ee476-06bb-4993-8156-78335a3c9163)

403 Forbidden: Forbidden: you must use a write token to upload to a repository..
Cannot access content at: https://huggingface.co/api/models/orhansonmeztr/qwen3-0.6b-4bit-turkish-mmlu-lora/preupload/main.
Make sure your token has the correct permissions.
